In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
from textblob import TextBlob
from nltk.corpus import stopwords
# import gensim
from nltk import sent_tokenize , word_tokenize
from gensim.utils import simple_preprocess
from tqdm import tqdm
# import contractions
import string

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Spam')

In [ ]:
df.isnull().sum()

,0
label,0
text,0


In [ ]:
x_train = df['text']
y_train = df['label']

In [ ]:
import nltk
nltk.download('stopwords')
def preprocess(x):
    x = x.str.lower()
    def remove_tags(t):
        pattern = re.compile('<.*?>')
        return pattern.sub(r'',t)
    x = x.apply(remove_tags)
    def remove_urls(text):
        url_pattern = re.compile(r'https?://\S+|www\.\S+')
        text_without_urls = url_pattern.sub('', text)
        return text_without_urls
    x = x.apply(remove_urls)
    lst = stopwords.words('english')
    x.apply(lambda xi:[item for item in xi.split() if item not in lst]).apply(lambda xi:" ".join(xi))
    TEXT_CLEANING_RE = "[^a-zA-z\s]" # Matches character that are not letter or whitespace to remove them
    def clean_text(text):
        text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip() # Remove unwanted characters
        cleaned_text = re.sub('\s+', ' ', text).strip() # Normalize whitespace
        return cleaned_text
    x = x.apply(lambda xi: clean_text(xi))
    return x

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
x_train = preprocess(x_train)

In [ ]:
x_train

,text
0,ounce feather bowl hummingbird opec moment ala...
1,wulvob get your medircations online qnb ikud v...
2,computer connection from cnn com wednesday esc...
3,university degree obtain a prosperous future m...
4,thanks for all your answers guys i know i shou...
...,...
83443,hi given a date how do i get the last date of ...
83444,now you can order software on cd or download i...
83445,dear valued member canadianpharmacy provides a...
83446,subscribe change profile contact us long term ...


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x_train , y_train , test_size = 0.3 )

In [ ]:
x_test ,x_val , y_test , y_val =  train_test_split(x_test , y_test , test_size = 0.5 )

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_features = 13000)

In [ ]:
x_train_tf = tf.fit_transform(x_train).toarray()
x_test_tf = tf.transform(x_test).toarray()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train_tf , y_train)
y_pred1 = gnb.predict(x_test_tf)
from sklearn.metrics import accuracy_score
accuracy_score(y_test , y_pred1)

0.9690021570663897

In [ ]:
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(x_train_tf, y_train)
y_pred3 = xg.predict(x_test_tf)
accuracy_score(y_test, y_pred3)

0.9796277063194057

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train_tf, y_train)
y_pred4 = dt.predict(x_test_tf)
accuracy_score(y_test, y_pred4)

0.9582967164656068

In [ ]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(x_train_tf, y_train)
y_pred5 = svm.predict(x_test_tf)
accuracy_score(y_test, y_pred5)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train_tf, y_train)
y_pred2 = rf.predict(x_test_tf)
accuracy_score(y_test , y_pred2)

0.9843412958376608

In [ ]:
import pickle

# Save the TfidfVectorizer
with open('tfidf_vectorizer2.pkl', 'wb') as f:
    pickle.dump(tf, f)

In [ ]:
with open("model3.pkl" , "wb") as f:
    pickle.dump(xg,f)